In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
# Ensure system packages are up-to-date
!apt-get update
!apt-get install -y git-lfs

# Clone the correct LangSAM repository
!rm -rf lang-segment-anything  # Remove any previous installation
!git clone https://github.com/luca-medeiros/lang-segment-anything.git
%cd lang-segment-anything

# Install dependencies
!pip install torch torchvision
!pip install -e .


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,724 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,735 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [ ]:
import os
import torch
from PIL import Image
import numpy as np
from lang_sam import LangSAM

# Set paths
input_folder = "/content/drive/My Drive/ECS 193/langsam/cnn_1/input"   # Folder containing input images
output_folder = "/content/drive/My Drive/ECS 193/langsam/cnn_1/output/" # Main output folder
segmented_folder = os.path.join(output_folder, "segmented_images") # Overlayed segmented images
mask_folder = os.path.join(output_folder, "masks") # Segmentation masks
prompt = "Red and white track boundary"  # Change this to your desired segmentation prompt

# Ensure input folder exists
if not os.path.exists(input_folder):
    raise FileNotFoundError(f"Input folder '{input_folder}' not found. Please upload images.")

# Ensure output subfolders exist (Create them if they don't)
os.makedirs(segmented_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# Initialize LangSAM model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LangSAM()

# Get all images from input folder
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not image_files:
    raise FileNotFoundError(f"No image files found in '{input_folder}'. Please upload images.")

# Process each image
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = Image.open(image_path).convert("RGB")

    # 🔹 Fix: Capture single output
    prediction = model.predict([image], [prompt])

    print(f"Prediction Output: {type(prediction)}")  # Debugging info
    print(prediction)  # Debugging info

    # Ensure prediction is a valid output
    if isinstance(prediction, list) and len(prediction) > 0:
        prediction_data = prediction[0]  # Extract first prediction dictionary

        # Extract masks correctly
        if 'masks' in prediction_data and prediction_data['masks'] is not None:
            masks = prediction_data['masks']

            if len(masks) > 0:
                mask_image = Image.fromarray((masks[0] * 255).astype(np.uint8))  # Convert first mask to grayscale
                mask_output_path = os.path.join(mask_folder, f"{os.path.splitext(image_file)[0]}_mask.png")
                mask_image.save(mask_output_path)

                # Overlay mask on original image
                overlay = np.array(image)
                overlay[masks[0] > 0] = [255, 0, 0]  # Highlight segmentation in red

                # Save overlayed segmented image
                overlay_image = Image.fromarray(overlay)
                overlay_output_path = os.path.join(segmented_folder, f"{os.path.splitext(image_file)[0]}_segmented.png")
                overlay_image.save(overlay_output_path)

                print(f"✅ Processed {image_file} - Mask saved at {mask_output_path}, Segmented image saved at {overlay_output_path}")
            else:
                print(f"⚠ No valid segmentation mask found for {image_file}")
        else:
            print(f"⚠ No 'masks' key in prediction output for {image_file}")
    else:
        print(f"⚠ Unexpected prediction format for {image_file}")

print("\n🎉 Processing complete! Check your output folder for results.")


/usr/local/lib/python3.11/dist-packages/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()
Downloading: "https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt" to /root/.cache/torch/hub/checkpoints/sam2.1_hiera_small.pt
100%|██████████| 176M/176M [00:00<00:00, 198MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/933M [00:00<?, ?B/s]

Predicting 1 masks


/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Streaming output truncated to the last 5000 lines.
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]]], dtype=float32), 'mask_scores': array(0.91015625, dtype=float32)}]
✅ Processed frame_0238_1747705557414688660.png - Mask saved at /content/drive/My Drive/ECS 193/langsam/cnn_1/output/masks/frame_0238_1747705557414688660_mask.png, Segmented image saved at /content/drive/My Drive/ECS 193/langsam/cnn_1/output/segmented_images/frame_0238_1747705557414688660_segmented.png
Predicting 1 masks
Predicted 1 masks
Prediction Output: <class 'list'>
[{'scores': array([0.31997073], dtype=float32), 'labels': ['red track boundary'], 'boxes': array([[569.7486 , 324.11432, 639.626  , 371.06027]], dtype=float32), 'masks': array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,